### Taxa de Aprendizado (Learning rate)

A taxa de aprendizado é um hiperparâmetro que controla o quanto os pesos da rede são ajustados em resposta ao gradiente calculado em cada iteração.

Considerações Importantes:

•	Programação da Taxa de Aprendizado: É comum usar uma taxa de aprendizado que diminui ao longo do tempo. No início do treinamento, uma taxa mais alta pode ajudar a rede a explorar a paisagem de perda de forma mais agressiva, mas conforme o treinamento progride, uma taxa mais baixa pode ajudar a refinar a solução encontrada, evitando que o algoritmo oscile em torno do mínimo.

•	Tentativa e Erro: A escolha da programação correta para a taxa de aprendizado (como ela diminui ao longo do tempo) geralmente requer experimentação. Técnicas como learning rate schedules (agendamentos de taxa de aprendizado) ou adaptive learning rates (taxas de aprendizado adaptativas) podem ser usadas para melhorar a convergência.

- Sobre: " **lr_scheduler = LearningRateScheduler(lr_schedule)**:

    - é usada para criar um callback que ajusta a taxa de aprendizado durante o treinamento do modelo. O Learning Rate Scheduler é uma técnica que ajusta dinamicamente a taxa de aprendizado (learning rate) ao longo do treinamento de uma rede neural. A taxa de aprendizado é um hiperparâmetro importante que determina o tamanho dos passos que o otimizador dá ao ajustar os pesos da rede com base no gradiente calculado.

    - Função LearningRateScheduler: é um tipo específico de callback no Keras que ajusta a taxa de aprendizado de acordo com uma função definida pelo usuário a cada época do treinamento.

- lr_scheduler = LearningRateScheduler(lr_schedule)

lr_schedule: Esta é a função que você define para especificar como a taxa de aprendizado deve ser ajustada ao longo do treinamento. 
A função lr_schedule(epoch) retorna diferentes valores para a taxa de aprendizado com base na época atual.

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Carregar o dataset MNIST
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Pré-processamento dos dados
x_train = x_train.astype('float32') / 256.0
x_test = x_test.astype('float32') / 256.0
x_train = np.expand_dims(x_train, axis=-1)
x_test = np.expand_dims(x_test, axis=-1)

# Transformar os rótulos em categorias (one-hot encoding)
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)


In [3]:
# Definir o modelo
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

c:\Users\Carlos Carneiro\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [4]:
# Definir uma função de programação da taxa de aprendizado. Escolha o valor de l_r
def lr_schedule(epoch):
    initial_lr = 0.001
    if epoch < 5:
        return initial_lr
    elif epoch < 10:
        return initial_lr * 0.5
    else:
        return initial_lr * 0.1


In [5]:
# Compilar o modelo com otimizador Adam e uma taxa de aprendizado inicial. Escolha o valor l_r
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Definir o callback de programação da taxa de aprendizado
lr_scheduler = LearningRateScheduler(lr_schedule)

In [6]:
# Executando 15 épocas. Tamanho dos mini lotes: 64 (64 amostras por vez antes de calcular o gradiente e ajustar os pesos).
# escolher isso também para melhorar o resultado de perda e acurácia.

# Treinar o modelo
history = model.fit(x_train, y_train, epochs=15, batch_size=64,
                    validation_data=(x_test, y_test),
                    callbacks=[lr_scheduler])

# Avaliar o modelo
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f'\nTest accuracy: {test_acc:.4f}')

# Plotar os gráficos de perda e acurácia
epochs = range(1, len(history.history['loss']) + 1)

plt.figure(figsize=(12, 5))

# Gráfico de perda
plt.subplot(1, 2, 1)
plt.plot(epochs, history.history['loss'], 'r-', label='Perda de Treino')
plt.plot(epochs, history.history['val_loss'], 'b-', label='Perda de Validação')
plt.title('Perda ao Longo das Épocas')
plt.xlabel('Épocas')
plt.ylabel('Perda')
plt.legend()

# Gráfico de acurácia
plt.subplot(1, 2, 2)
plt.plot(epochs, history.history['accuracy'], 'r-', label='Acurácia de Treino')
plt.plot(epochs, history.history['val_accuracy'], 'b-', label='Acurácia de Validação')
plt.title('Acurácia ao Longo das Épocas')
plt.xlabel('Épocas')
plt.ylabel('Acurácia')
plt.legend()

plt.show()


Epoch 1/15
938/938 ━━━━━━━━━━━━━━━━━━━━ 20s 20ms/step - accuracy: 0.8796 - loss: 0.4044 - val_accuracy: 0.9852 - val_loss: 0.0472 - learning_rate: 0.0010
Epoch 2/15
320/938 ━━━━━━━━━━━━━━━━━━━━ 11s 19ms/step - accuracy: 0.9849 - loss: 0.0521